<a href="https://colab.research.google.com/github/abdulraheem-hammad/AI-Grading-System/blob/main/AI_Grading_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.6/265.6 kB 11.3 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.5.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import json
from datasets import Dataset

# Load the JSON file
file_path = "questions.json"
with open(file_path, 'r', encoding="utf-8") as file:
    data = json.load(file)
    dataset = data["question"]  # Extract the list of question-answer pairs

#
def convert_to_conversation(sample):
    conversation = [
        {
            "from": "system",
            "value": "You are a helpful assistant that provides correct answers to questions"
        },
        {
            "from": "human",
            "value": sample['question']
        },
        {
            "from": "gpt",
            "value": sample['answer']
        }
    ]
    return {"conversations": conversation}

# Convert each sample to conversation format
data_temp = [convert_to_conversation(data) for data in dataset]

# Create the data dictionary
data_dict = {
    "conversations": [item["conversations"] for item in data_temp]
}


dataset = Dataset.from_dict(data_dict)

# Verify the result
print(f"Loaded {len(dataset)} conversations")
print("First conversation:", dataset[0])

Loaded 1031 conversations
First conversation: {'conversations': [{'from': 'system', 'value': 'You are a helpful assistant that provides correct answers to questions'}, {'from': 'human', 'value': 'What is a distributed system?'}, {'from': 'gpt', 'value': 'A distributed system is a collection of independent computers that work together to appear as a single coherent system.'}]}


In [ ]:
from unsloth.chat_templates import standardize_sharegpt
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/1031 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map:   0%|          | 0/1031 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, #
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,

        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1031 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,031 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,4.491200
2,4.478900
3,4.176400
4,3.526400
5,2.962500
6,2.523500
7,1.886600
8,1.498600
9,1.246900
10,0.924700


In [ ]:
FastLanguageModel.for_inference(model)

# Define the system message as a string
system_message = """
You are a helpful assistant that evaluates answers by comparing them to the correct answer. Your task is to follow these steps:
1) Compare the given answer to the correct answer, noting similarities and differences.
2) Determine if the given answer is fully correct, partially correct, or incorrect, and explicitly state your determination.
3) Provide a detailed explanation of your reasoning, confirming any correct parts of the given answer and explaining any errors or omissions.
"""

# Define the user message as a string
user_message = """
Question: describe in details the software development life cycle, with exmaple and justify your answer  .
Correct Answer: The Software Development Life Cycle (SDLC) is a structured process used to develop software applications. It typically involves several phases, including planning, analysis, design, implementation, testing, deployment, and maintenance. Each phase has specific tasks and deliverables, ensuring a systematic approach to software development.
Here's a detailed look at each phase:
1. Planning:
This initial phase involves defining the project's goals, scope, and requirements.
Key activities include identifying stakeholders, conducting feasibility analysis, and creating a project plan.
The goal is to establish a clear understanding of the project's objectives and constraints.
2. Analysis:
This phase focuses on gathering and understanding the specific requirements of the software.
Activities include user interviews, document analysis, and prototyping.
The goal is to accurately capture the needs and expectations of the stakeholders.
3. Design:
Based on the analysis phase, the design phase involves creating the software's architecture, components, and user interface.
Activities include developing data models, designing the user interface, and selecting technologies.
The goal is to create a blueprint for the software that meets the identified requirements.
4. Implementation (Coding):
This phase involves translating the design into actual code.
Activities include coding, debugging, and unit testing.
The goal is to create a functional version of the software.
5. Testing:
The testing phase involves verifying that the software meets the specified requirements and is free from defects.
Activities include unit testing, integration testing, system testing, and user acceptance testing.
The goal is to ensure the software's quality and reliability.
6. Deployment:
This phase involves releasing the tested software to end users.
Activities include preparing the software for deployment, installing it on the target systems, and providing user documentation.
The goal is to make the software available to users.
7. Maintenance:
The maintenance phase involves addressing defects, providing updates, and ensuring the software's long-term viability.
Activities include bug fixing, code enhancements, and performance optimization.
The goal is to keep the software running smoothly and up-to-date..
Evaluate the following answers:
Answer a1:sdlc is about planning : montoring then testing then vervication and valcation.
For each answer, follow these steps:
Step 1: Compare the given answer to the correct answer, noting similarities and differences.
Step 2: Determine if the given answer is fully correct, partially correct, or incorrect, and explicitly state your determination.
Step 3: Provide a detailed explanation of your reasoning, confirming any correct parts of the given answer and explaining any errors or omissions.
step 4: give a final score of 5, if the answer is fully correct assign 5, if its partilly correct give 3, if it's wrong give 0
"""


messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]


inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 0.1, min_p = 0.1)

Step 1: The given answer mentions planning, monitoring, testing, verification, and validation, which are indeed part of the SDLC. However, it lacks a clear description of the phases and their order.

Step 2: The given answer is partially correct.

Step 3: The correct SDLC phases are planning, analysis, design, implementation, testing, deployment, and maintenance. The given answer mentions some of these phases but does not provide a clear and accurate description.

Step 4: Final score: 3<|eot_id|>


In [ ]:
# Define the system message with clear instructions for the model
system_message = """
You are a helpful assistant that evaluates answers by comparing them to the correct answer. Your task is to follow these steps:
1) Compare the given answer to the correct answer, noting similarities and differences.
2) Determine if the given answer is fully correct, partially correct, or incorrect, and explicitly state your determination.
3) Provide a detailed explanation of your reasoning, confirming any correct parts of the given answer and explaining any errors or omissions.
"""

# Define the question and correct answer
question = "What is Openness in distributed systems?"
correct_answer = "Openness in distributed systems refers to the ability to extend and improve the system by integrating new components with existing ones, while resolving differences in data representation across different processors."

# File paths
input_file = "student_answers.txt"
output_file = "graded_student_answers.txt"

with open(input_file, "r", encoding="utf-8") as infile:
    lines = infile.readlines()

with open(output_file, "w", encoding="utf-8") as outfile:
    outfile.write(f"Question: {question}\n\n")
    outfile.write(f"Correct Answer: {correct_answer}\n\n")
    outfile.write("---\n\n")

    for line in lines:
        if ":" in line:
            student_name, answer_to_evaluate = line.split(":", 1)
            student_name = student_name.strip()
            answer_to_evaluate = answer_to_evaluate.strip()

            user_message = f"""
Question: {question}
Correct Answer: {correct_answer}
Evaluate the following answer:
Answer: {answer_to_evaluate}
"""

            messages = [
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ]

            inputs = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to("cuda")

            generated_ids = model.generate(
                input_ids=inputs,
                max_new_tokens=128,
                use_cache=True,
                temperature=0.1,
                min_p=0.1
            )

            generated_text = tokenizer.decode(generated_ids[0, inputs.shape[1]:], skip_special_tokens=True)

            outfile.write(f"Student: {student_name}\n")
            outfile.write(f"Answer: {answer_to_evaluate}\n")
            outfile.write(f"Evaluation: {generated_text}\n")
            outfile.write("---\n\n")

print(f"Evaluations have been written to {output_file}")


Evaluations have been written to graded_student_answers.txt
